In [1]:
import base64
import json
import os
from pathlib import Path
import time
import PIL

import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from vllm import LLM

from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    model_config = SettingsConfigDict(
        env_file="../.env", env_file_encoding="utf-8", extra="ignore"
    )
    model_dir: str
    
settings = Settings()

INFO 01-30 21:06:47 __init__.py:183] Automatically detected platform cuda.


In [2]:
model_cache_dir = settings.model_dir
model_dir = os.path.join(model_cache_dir, "Qwen2-VL-2B")
# model_dir = os.path.join(settings.model_dir, "Qwen2.5-VL-3B-Instruct") # 2.5 not supported yet

llm = LLM(
    model=model_dir,
    trust_remote_code=True,
    max_model_len=8192,
    limit_mm_per_prompt={"image": 4},
    task="embed",
    device="cuda"
)

INFO 01-30 21:06:55 llm_engine.py:232] Initializing an LLM engine (v0.7.0) with config: model='/home/users/yrsong/llm_serving/models/Qwen2-VL-2B', speculative_config=None, tokenizer='/home/users/yrsong/llm_serving/models/Qwen2-VL-2B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/home/users/yrsong/llm_serving/models/Qwen2-VL-2B, num_scheduler_steps=1, multi_step_stream_outputs=True, enable_prefix_caching=False, chunked_prefill_

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


WARNING 01-30 21:07:21 utils.py:173] Unable to collect loaded parameters for module Qwen2ForEmbedding(
WARNING 01-30 21:07:21 utils.py:173]   (model): Qwen2Model(
WARNING 01-30 21:07:21 utils.py:173]     (embed_tokens): VocabParallelEmbedding(num_embeddings=151936, embedding_dim=1536, org_vocab_size=151936, num_embeddings_padded=151936, tp_size=1)
WARNING 01-30 21:07:21 utils.py:173]     (layers): ModuleList(
WARNING 01-30 21:07:21 utils.py:173]       (0-27): 28 x Qwen2DecoderLayer(
WARNING 01-30 21:07:21 utils.py:173]         (self_attn): Qwen2Attention(
WARNING 01-30 21:07:21 utils.py:173]           (qkv_proj): QKVParallelLinear(in_features=1536, output_features=2048, bias=True, tp_size=1, gather_output=False)
WARNING 01-30 21:07:21 utils.py:173]           (o_proj): RowParallelLinear(input_features=1536, output_features=1536, bias=False, tp_size=1, reduce_results=True)
WARNING 01-30 21:07:21 utils.py:173]           (rotary_emb): MRotaryEmbedding(head_size=128, rotary_dim=128, max_pos

In [17]:
# https://docs.vllm.ai/en/stable/serving/multimodal_inputs.html#embedding
# image placeholders: https://github.com/huggingface/transformers/blob/365fecb4d0b6c87f20b93561e11c3d4c77938012/src/transformers/models/qwen2_vl/processing_qwen2_vl.py#L63
# processor replaces `<|image_pad|>`  with image data
text = "시간표: <|image_pad|><|image_pad|>"
image1 = PIL.Image.open("resources/finance-sample/table-1.png")

# https://github.com/vllm-project/vllm/blob/a2769032ca78108e58abc45e2eb0ade8b47a6515/vllm/model_executor/models/qwen2_vl.py#L947
prompt = {
    "prompt": text,
    "multi_modal_data": {
        "image": [image1, image1]
    },
}

In [18]:
outputs = llm.embed(prompt)
print(type(outputs))

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.79it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

<class 'list'>


In [23]:
embedding = outputs[0].outputs.embedding
print(len(embedding))


1536
